In [1]:
import os

# Change the working directory
os.chdir("../")
HOME = os.getcwd()
print(f"Current working directory: {HOME}")

Current working directory: D:\Crude_Oil_Price_Prediction____Color Sketch\Crude_Oil_Price_Prediction\crude_oil_price_prediction


#### Add the component configuration in config/config.yaml

#### Update the entity

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict
    
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

#### Update configuraion manager in src/mlProject/config/configuration

In [26]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger

from mlProject.entity.config_entity import DataIngestionConfig, DataValidationConfig, DataValidationConfig
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        return data_validation_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataValidationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config

In [25]:
config = ConfigurationManager()

[2024-04-09 23:41:14,765: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-09 23:41:14,765: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-09 23:41:14,765: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-09 23:41:14,781: INFO: common: created directory at: artifacts]


In [23]:
data_transformation_config = config.get_data_transformation_config()

[2024-04-09 23:39:30,611: INFO: common: created directory at: artifacts/data_transformation]


TypeError: DataValidationConfig.__init__() got an unexpected keyword argument 'data_path'

#### Update the component

mlProject/components/data_transformation.py

In [13]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from mlProject import logger

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        
        # Read the csv file data 
        data = pd.read_csv(self.config.data_path)
        
        # Split the data into training and test sets. (0.75, 0.25)
        train, test = train_test_split(data, test_size=0.25)

        # Save train.csv and test.csv into artifacts/data_transformation
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splited data into training and test sets.")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

#### Update the Pipeline

mlProject/pipeline/stage_03_data_transformation.py

In [1]:
import os
os.chdir("../")

from mlProject.config.configuration import ConfigurationManager
from mlProject.components.data_transformation import DataTransformation
from mlProject import logger
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    logger.exception(e)
    raise e

[2024-04-09 23:57:46,962: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-09 23:57:46,964: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-09 23:57:46,964: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-09 23:57:46,964: INFO: common: created directory at: artifacts]
[2024-04-09 23:57:46,964: INFO: common: created directory at: artifacts/data_transformation]
[2024-04-09 23:57:47,145: INFO: data_transformation: Splited data into training and test sets.]
[2024-04-09 23:57:47,145: INFO: data_transformation: (1199, 12)]
[2024-04-09 23:57:47,145: INFO: data_transformation: (400, 12)]


In [3]:
import os
os.getcwd()

'D:\\Crude_Oil_Price_Prediction____Color Sketch\\Crude_Oil_Price_Prediction\\crude_oil_price_prediction\\research'

In [2]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()

[2024-04-09 23:54:14,573: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-09 23:54:14,573: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-09 23:54:14,589: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-09 23:54:14,590: INFO: common: created directory at: artifacts]
[2024-04-09 23:54:14,591: INFO: common: created directory at: artifacts/data_ingestion]


In [3]:
data_validation_config =config.get_data_validation_config()

[2024-04-09 23:54:35,855: INFO: common: created directory at: artifacts/data_validation]


In [4]:
data_transformation_config = config.get_data_transformation_config()

[2024-04-09 23:54:54,804: INFO: common: created directory at: artifacts/data_transformation]


TypeError: DataValidationConfig.__init__() got an unexpected keyword argument 'data_path'

In [5]:
!python ./src/mlProject/pipeline/stage_03_data_transformation.py

[2024-04-10 00:07:31,621: INFO: stage_03_data_transformation: >>>>>>>> stage Data Transformation stage started <<<<<<<<]
[2024-04-10 00:07:31,621: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-10 00:07:31,621: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-10 00:07:31,621: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-10 00:07:31,621: INFO: common: created directory at: artifacts]
[2024-04-10 00:07:31,621: INFO: common: created directory at: artifacts/data_transformation]
[2024-04-10 00:07:31,653: INFO: data_transformation: Splited data into training and test sets.]
[2024-04-10 00:07:31,653: INFO: data_transformation: (1199, 12)]
[2024-04-10 00:07:31,653: INFO: data_transformation: (400, 12)]
[2024-04-10 00:07:31,653: INFO: stage_03_data_transformation: >>>>>>>> stage Data Transformation stage completed <<<<<<<<

X==================sX]


In [7]:
!python main.py

[2024-04-10 00:18:50,729: INFO: main: >>>>>>>> stage Data Ingestion stage started <<<<<<<<]
[2024-04-10 00:18:50,737: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-10 00:18:50,737: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-10 00:18:50,739: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-10 00:18:50,739: INFO: common: created directory at: artifacts]
[2024-04-10 00:18:50,739: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-10 00:18:51,486: INFO: data_ingestion: artifacts/data_ingestion/data.zip downloaded! with following info Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mo

Traceback (most recent call last):
  File "d:\crude_oil_price_prediction____color sketch\crude_oil_price_prediction\crude_oil_price_prediction\src\mlProject\pipeline\stage_03_data_transformation.py", line 25, in main
    raise Exception("Your data schema is not valid.")
Exception: Your data schema is not valid.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Crude_Oil_Price_Prediction____Color Sketch\Crude_Oil_Price_Prediction\crude_oil_price_prediction\main.py", line 44, in <module>
    raise e
  File "D:\Crude_Oil_Price_Prediction____Color Sketch\Crude_Oil_Price_Prediction\crude_oil_price_prediction\main.py", line 40, in <module>
    obj.main()
  File "d:\crude_oil_price_prediction____color sketch\crude_oil_price_prediction\crude_oil_price_prediction\src\mlProject\pipeline\stage_03_data_transformation.py", line 28, in main
    raise e
NameError: name 'e' is not defined. Did you mean: 'es'?
